In [ ]:
import pandas as pd
import os
import numpy as np

In [ ]:
#working directory
working_dir = "/Users/augustus/Desktop/WSWC/WaDE/MappingStatesDataToWaDE2.0/Wyoming/AggregatedAmounts/RawInputData/"
os.chdir(working_dir)

In [ ]:
# create output dataframe
target_columns = ["ReportingUnitUUID", "ReportingUnitNativeID", "ReportingUnitName",
                  "ReportingUnitTypeCV", "ReportingUnitUpdateDate", "ReportingUnitProductVersion",
                  "StateCV", "EPSGCodeCV", "Geometry"]

outdf = pd.DataFrame(columns=target_columns)

In [ ]:
# input file
file = "Agricultural, Municipal, Domestic, and Industrial Use Indecies_Draft_For Delivery to WWDC_2019.0703.xlsx"

In [ ]:
# import file to dataframe
df = pd.read_excel(file, header=2)
df = df.rename(columns={'Unnamed: 1':'UseType', 'Unnamed: 2':'Source'})

# Fill nan with 'blank'
df= df.fillna('blank')

In [ ]:
# distribute basin names over blanks
x = df.loc[0, 'Water Use by Basin']
for i, row in df.iterrows():
    y = row['Water Use by Basin']
    if y == 'blank':
        df.at[i, 'Water Use by Basin'] = x
    else:
        x = y

In [ ]:
# only need county names in reporting units
# remove redundancies
df = df.drop_duplicates(subset='Water Use by Basin')
df = df.reset_index(drop=True)

#Todo: ask Adel about deleting statewide row 
indexNames = df[ (df['Water Use by Basin'] == 'blank') | (df['Water Use by Basin'] == 'Statewide Water Use Index')].index
df.drop(indexNames , inplace=True)

outdf["ReportingUnitName"] = df["Water Use by Basin"]

In [ ]:
# Reporting unit native ID:  Auto generate
outdf['ReportingUnitNativeID'] = range(1, len(outdf.index) + 1)

# ReportingUnitUUID:  WY_NativeID
outdf['ReportingUnitUUID'] = outdf.apply(lambda row: "_".join(["WY", str(row['ReportingUnitNativeID'])]), axis=1)

In [ ]:
# hardcoded
outdf.StateCV = "WY"
outdf.ReportingUnitTypeCV = "Basin"

#ToDo:  get hardcoded EPSG code from Adel
#outdf.EPSGCodeCV = "EPSG:4326"

# replace NaN with blank cells
outdf = outdf.replace(np.nan, '')
outdf.head(5)

In [ ]:
print("Exporting csv...")


#write out
out_repunit = 'reportingunits.csv'
outdf.to_csv(out_repunit, index=False, encoding = "utf-8")


print("DONE")